<a href="https://colab.research.google.com/github/meti-94/TextGeneration/blob/main/personachat_simple_trainer_final_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
!tar -xf  simple-examples.tgz
!mkdir data
!mv /content/simple-examples/data/ptb.train.txt data/
!mv /content/simple-examples/data/ptb.valid.txt data/
!mv /content/simple-examples/data/ptb.test.txt data/
!rm -rf ./simple_examples

In [1]:
!pip install "ray[tune]" transformers datasets==1.1.2
# !pip install transformers
!ray install-nightly
!pip install rouge_score
!pip install -U ray
# !pip install -U transformers datasets
!pip install tensorboardX


     |████████████████████████████████| 49.7MB 63kB/s 
     |████████████████████████████████| 2.1MB 28.7MB/s 
     |████████████████████████████████| 153kB 55.5MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 133kB 43.6MB/s 
     |████████████████████████████████| 1.3MB 45.7MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
     |████████████████████████████████| 1.0MB 44.1MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 3.1MB 52.0MB/s 
     |████████████████████████████████| 122kB 58.5MB/s 
     |████████████████████████████████| 3.3MB 47.7MB/s 
     |████████████████████████████████| 901kB 46.7MB/s 
     |████████████████████████████████| 245kB 58.2MB/s 
     |████████████████████████████████| 296kB 56.8MB/s 
     |████████████████████████████████| 143kB 58.4MB/s 
     |████████████████████████████████| 92kB 11.5MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
Running: /usr/bin/python3 -m pip install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-2.0.0.dev0-cp37-cp37m-manylinux2014_x86_64.whl.
     |████████████████████████████████| 49.5MB 65kB/s 
  Found existing installation: ray 1.3.0
    Uninstalling ray-1.3.0:
      Successfully uninstalled ray-1.3.0
Requirement already up-to-date: ray in /usr/local/lib/python3.7/dist-packages (2.0.0.dev0)


In [ ]:
import transformers
transformers.__version__

'4.5.1'

In [2]:
!wget https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
!mkdir models

--2021-05-07 12:55:01--  https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.46.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.46.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209850483 (200M) [application/json]
Saving to: ‘personachat_self_original.json’

personachat_self_or 100%[===================>] 200.13M  61.4MB/s    in 3.3s    

2021-05-07 12:55:05 (61.4 MB/s) - ‘personachat_self_original.json’ saved [209850483/209850483]



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
# import ray
# from ray.tune.suggest.hyperopt import HyperOptSearch
# from ray.tune.schedulers import AsyncHyperBandScheduler



from transformers import EncoderDecoderModel, BertTokenizer, BertModel, BertConfig
from typing import Any, Dict, List, Optional, Tuple, Union
import torch
from torch.utils.data import Dataset
from transformers import AdamW
from tqdm import tqdm
import json
import random 
from sklearn.model_selection import train_test_split
from datetime import datetime
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from tqdm import tqdm_notebook as tqdm
import logging
import datasets
logging.basicConfig(level=logging.DEBUG)


global print_threshold, counter
print_threshold = 10
counter = 0
# from customize.trainer_customization import CustomSeq2SeqTrainer

In [27]:
SPECIAL_TOKENS = ["<bos>", "<eos>", "<persona>", "<speaker1>", "<speaker2>", "<pad>"]

ATTR_TO_SPECIAL_TOKEN = {'bos_token': '<bos>', 'eos_token': '<eos>', 'pad_token': '<pad>',
                         'additional_special_tokens': ['<speaker1>', '<speaker2>', '<persona>']}

encoder_max_length=512
decoder_max_length=128

def read_data(data_json_path='/content/personachat_self_original.json'):
  with open(data_json_path) as json_file:
    data_dict = json.load(json_file)
  return data_dict

def data_to_samples(data_dict, test=False):
  samples=[]
  for dialogue in (data_dict['train'] if test==False else data_dict['valid']):
    original_persona = dialogue['personality']
    
    for item in dialogue['utterances']:
      original_persona = [original_persona[-1]] + original_persona[:-1]
      history = item['history']
      response = item['candidates'][-1]
      samples.append({
          'persona':original_persona,
          'history':history,
          'response':response
      })
  return samples
  
def bertified(samples):
  bertified_data = []
  for item in samples:
    persona = ' <persona> '.join(item['persona'])
    persona = '<bos> ' + persona
    history = ''
    speakers = [" <speaker1> ", " <speaker2> "]
    speaker = 0
    for hst in item['history'][::-1]:
      history = speakers[speaker] + hst + history
      speaker = 1 - speaker
    response = '<speaker2> ' + item['response'] + ' <eos>'
    bertified_data.append({
          'persona':persona.replace('  ', ' '),
          'history':history.replace('  ', ' '),
          'input': persona.replace('  ', ' ')+' '+history.replace('  ', ' '), 
          'response':response.replace('  ', ' ')
      })
  return bertified_data

def bertified_to_model_food(bertified_data, tokenizer):
    model_food = []
    pbar = tqdm(bertified_data)
    pbar.set_description('Tokenizing Data Sample ...')
    for item in pbar:
        inputs = tokenizer(item['input'], add_special_tokens=True, max_length=encoder_max_length, truncation=True, padding="max_length")
        outputs = tokenizer(item['response'], add_special_tokens=True, max_length=decoder_max_length, truncation=True, padding="max_length")
        _item = {}
        _item["input_ids"] = inputs.input_ids
        _item["attention_mask"] = inputs.attention_mask
        _item["decoder_input_ids"] = outputs.input_ids
        _item["decoder_attention_mask"] = outputs.attention_mask
        _item["labels"] = outputs.input_ids.copy()
        model_food.append(_item)
    return model_food 

In [28]:
class PersonaDataset_v3(Dataset):
  '''
      Convert Data to proper Tensor dataset
  '''
  def __init__(self, samples):
    self.samples = samples
    self.n_samples = len(self.samples)

  def __getitem__(self, index):
    # returns specific item
    return self.samples[index]

  def __len__(self):
    return self.n_samples

In [29]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("prajjwal1/bert-tiny", "prajjwal1/bert-tiny")
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
tokenizer.add_special_tokens(ATTR_TO_SPECIAL_TOKEN)
model.get_encoder().resize_token_embeddings(len(tokenizer))
model.get_decoder().resize_token_embeddings(len(tokenizer))

data = read_data('./personachat_self_original.json')
data_samples = data_to_samples(data)
bertified_data = bertified(data_samples)
train, valid = train_test_split(bertified_data, test_size=0.15, random_state=99)
train = bertified_to_model_food(train, tokenizer)
valid = bertified_to_model_food(valid, tokenizer)
test_data = read_data('./personachat_self_original.json')
test_data_samples = data_to_samples(test_data, True)
test = bertified(test_data_samples)
test = bertified_to_model_food(test, tokenizer)



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationsh

In [30]:
# train_dataset = PersonaDataset_v3(list(zip(*map(one_line_tokenizer, tqdm(train)))))
train_dataset = PersonaDataset_v3(train)
valid_dataset = PersonaDataset_v3(valid)
test_dataset = PersonaDataset_v3(test)



In [31]:
rouge = datasets.load_metric("rouge")
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/metrics/rouge/rouge.py HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/1.1.2/metrics/rouge/rouge.py HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "HEAD /huggingface/datasets/1.1.2/metrics/rouge/dataset_infos.json HTTP/1.1" 404 0
DEBUG:filelock:Attempting to acquire lock 139684016067344 on /root/.cache/huggingface/datasets/336a0cbdd138e0d6e1361544ed2c56565192be45b5398cd55ac988d4b3fd1455.a89f1fa0750909f2d149b1ecabc808fb66cb865c94bb8bbb135c55deb50da2d7.py.lock
INFO:filelock:Lock 139684016067344 acquired on /root/.cache/hugging

In [32]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

# model.config.max_length = 142
# model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [33]:
from torch import nn
from transformers.trainer_utils import PredictionOutput
import random 


class CustomSeq2SeqTrainer(Seq2SeqTrainer):
    def evaluate(
        self,
        eval_dataset: Optional[Dataset] = None,
        ignore_keys: Optional[List[str]] = None,
        metric_key_prefix: str = "eval",
        max_length: Optional[int] = 128,
        num_beams: Optional[int] = None,
    ) -> Dict[str, float]:
        return super().evaluate(eval_dataset, ignore_keys=ignore_keys, metric_key_prefix=metric_key_prefix)
    def prediction_step(
        self,
        model: nn.Module,
        inputs: Dict[str, Union[torch.Tensor, Any]],
        prediction_loss_only: bool,
        ignore_keys: Optional[List[str]] = None,
    ) -> Tuple[Optional[float], Optional[torch.Tensor], Optional[torch.Tensor]]:
        global counter, print_threshold
        (loss, logits, labels) = super().prediction_step(model, inputs, prediction_loss_only, ignore_keys)
        if counter<print_threshold:
          random_subset = random.sample(range(0, len(inputs['labels'])), 10)
          for item in random_subset:
              persona_history_input = self.tokenizer.batch_decode([inputs['input_ids'][item]])
              real_output = self.tokenizer.batch_decode([inputs['labels'][item]])
              model_output = self.tokenizer.batch_decode([logits[item]])
              print("History & Persona & Input: ", persona_history_input[0].replace("<pad>", ""))
              print("Input: ", real_output[0].replace("<pad>", ""))
              print("Output: ", model_output[0].replace("<pad>", ""))
              counter+=1
        return loss, logits, labels
        

In [34]:
steps = 10
batch_size = 64
training_args = Seq2SeqTrainingArguments(
    output_dir="./models/",
    do_eval=True,
    do_predict =True,

    
    evaluation_strategy="steps",
    save_strategy = 'steps',
    
    eval_steps=steps,
    logging_steps=steps,
    save_steps=steps,
    
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    fp16=True, 
    
    
    save_total_limit = 2,
)

# instantiate trainer
def model_init():
    return model


trainer = CustomSeq2SeqTrainer(
    model_init=model_init,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
#     test_dataset=test_dataset,
    tokenizer=tokenizer,
#     max_length = 128
)




In [35]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure,Runtime,Samples Per Second
10,8.937200,4.626111,0.000000,0.000000,0.000000,1982.985800,9.943000


History & Persona & Input:  [CLS] <bos> i m retired. <persona> i like cooking healthy meals. <persona> i like to go to the gym and work out. <persona> i like to go out to the bars. <speaker1> _ _ silence _ _ <speaker2> hello, how are you today? <speaker1> i am pretty bored today, being nostalgic. [SEP]                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
Input:  [CLS] <speaker2> i hear you! maybe i'll go out to a bar tonight. <eos> [SEP]                                                                                                             
Output:  [CLS] ) ( ( air air ) air air air - air air

DEBUG:filelock:Attempting to acquire lock 139683292404176 on /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
INFO:filelock:Lock 139683292404176 acquired on /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
DEBUG:filelock:Attempting to release lock 139683292404176 on /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
INFO:filelock:Lock 139683292404176 released on /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
DEBUG:filelock:Attempting to acquire lock 139683414276944 on /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
INFO:filelock:Lock 139683414276944 acquired on /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
INFO:/usr/local/lib/python3.7/dist-packages/datasets/metric.py:Removing /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow
DEBUG:filelock:Attempting to release lock 

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure,Runtime,Samples Per Second
10,8.937200,4.626111,0.000000,0.000000,0.000000,1982.985800,9.943000
20,3.554100,1.402011,0.000400,0.000400,0.000400,1989.761000,9.909000


DEBUG:filelock:Attempting to acquire lock 139683412507472 on /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
INFO:filelock:Lock 139683412507472 acquired on /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
DEBUG:filelock:Attempting to release lock 139683412507472 on /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
INFO:filelock:Lock 139683412507472 released on /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
DEBUG:filelock:Attempting to acquire lock 139683406860112 on /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
INFO:filelock:Lock 139683406860112 acquired on /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow.lock
INFO:/usr/local/lib/python3.7/dist-packages/datasets/metric.py:Removing /root/.cache/huggingface/metrics/rouge/default/default_experiment-1-0.arrow
DEBUG:filelock:Attempting to release lock 

KeyboardInterrupt: ignored

In [ ]:
# from ray import tune
# from ray.tune.suggest.hyperopt import HyperOptSearch
# from ray.tune.schedulers import AsyncHyperBandScheduler

trainer.hyperparameter_search(
    direction="maximize", 
    backend="ray", 
    n_trials=4, # number of trials
)

== Status ==
Memory usage on this node: 11.0/31.3 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/12 CPUs, 1.0/2 GPUs, 0.0/15.6 GiB heap, 0.0/7.8 GiB objects (0.0/1.0 accelerator_type:GTX)
Result logdir: /home/textmining/ray_results/_objective_2021-04-23_16-10-57
Number of trials: 4/4 (3 PENDING, 1 RUNNING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+----------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------+----------|
| _objective_c4b35_00000 | RUNNING  |       |     5.61152e-06 |                  5 |                            64 |  8.15396 |
| _objective_c4b35_00001 | PENDING  |       |     1.56207e-05 |                  2 |                            16 |  7.08379 |
| _objective_c4b35_00002 | PENDING  |

(pid=17964) /home/textmining/anaconda3/envs/chatbot/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
(pid=17964)   "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>


(pid=18007) /home/textmining/anaconda3/envs/chatbot/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
(pid=18007)   "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <

...

(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <

(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <

2021-04-23 17:00:15,347	WARNING tune.py:507 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <IPython.core.display.HTML object>
(pid=17964) <IPython.core.display.HTML object>
(pid=18007) <

KeyboardInterrupt: 